In [ ]:
import os
import json
from PIL import Image
import numpy as np
from pycocotools import mask 
from skimage import measure
from itertools import groupby
import cv2

CATEGORIES: dict[str, int] = {
    "SA": 1,
    "LI": 2,
    "RI": 3,
}

def _shift(category_id: int, fragment_id: int) -> int:
    return 10 * (category_id - 1) + fragment_id

def load_masks(path) -> tuple[np.ndarray, list[int], list[int]]:
    seg = np.array(Image.open(path))
    return seg_to_masks(seg)

def seg_to_masks(seg: np.ndarray) -> tuple[np.ndarray, list[int], list[int]]:
    """Convert a binary-encoded multi-label segmentation to masks."""
    category_ids = []
    fragment_ids = []
    masks = []
    for category_id in CATEGORIES.values():
        for fragment_id in range(1, 11):
            mask = np.right_shift(seg, _shift(category_id, fragment_id)) & 1
            if mask.sum() > 0:
                masks.append(mask.astype('uint8'))
                category_ids.append(category_id)
                fragment_ids.append(fragment_id)

    return np.array(masks), category_ids, fragment_ids

def binary_mask_to_polygon(binary_mask):
    # binary_mask = binary_mask >
    # binary_mask = binary_mask > 0.5
    # binary_mask = binary_mask.astype(np.uint8)

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for contour in contours:
        # if contour.size >= 6:
        polygon = contour.flatten().tolist()
        polygons.append(polygon)
    return polygons

def create_coco_annotation(img_id, annotation_id, category_id, binary_mask, image_size):
    fortran_ground_truth_binary_mask = np.asfortranarray(binary_mask)
    encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
    ground_truth_area = mask.area(encoded_ground_truth)
    ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
    # contours = measure.find_contours(binary_mask, 0.5)


    annotation = {
            "segmentation":binary_mask_to_polygon(binary_mask),
            "area": ground_truth_area.tolist(),
            "iscrowd": 0,
            "image_id": img_id,
            "bbox": ground_truth_bounding_box.tolist(),
            "category_id": category_id,
            "id": annotation_id
        }
    return annotation

def convert_to_coco_format(img_dir, ann_dir, output_file):
    coco_dataset = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # category_id = 1
    categories = [{"id": 1, "name": "SA"} , {'id' : 2 , 'name': 'LI'} , {'id':3 , 'name': 'RI'}]
    coco_dataset["categories"] = categories

    annotation_id = 1
    img_id = 1
    files = os.listdir(img_dir)
    files.sort()
    files = files[2800:2900] + files[5100 : 5200] + files[7600 : 7700] + files[16600:16700]
    for img_filename in files:
        img_path = os.path.join(img_dir, img_filename)
        img = Image.open(img_path)
        width, height = img.size
        
        image_info = {
            "file_name": img_filename,
            "height": height,
            "width": width,
            "id": img_id
        }
        coco_dataset["images"].append(image_info)
                
        binary_masks , category_ids , fragment_ids = load_masks(os.path.join(ann_dir, img_filename) )

        for binary_mask , category_id ,fragment_id in zip(binary_masks , category_ids , fragment_ids):
            annotation = create_coco_annotation(img_id, annotation_id, category_id, binary_mask, (width, height))
            print(annotation)
            coco_dataset["annotations"].append(annotation)
            annotation_id += 1
        img_id += 1
        print(img_id)

    with open(output_file, 'w') as f:
        json.dump(coco_dataset, f, indent=4)

# Paths to your image and annotation directories
img_dir = '/scratch/dr/y.nawar/pengwin/train/input/images/x-ray/'
ann_dir = '/scratch/dr/y.nawar/pengwin/train/output/images/x-ray/'
output_file = 'coco_annotations_new.json'

convert_to_coco_format(img_dir, ann_dir, output_file)


In [ ]:
import base64
import IPython
import json
import numpy as np
import os
import random
import requests
from io import BytesIO
from math import trunc
from PIL import Image 
from PIL import ImageDraw as PILImageDraw
from pengwin_utils import visualize_drr
import os
import json
from PIL import Image
import numpy as np
from pycocotools import mask 
from skimage import measure
from itertools import groupby
import cv2

def apply_translucent_mask(image, mask, color, alpha=0.5):
    colored_mask = np.zeros_like(image)
    colored_mask[mask == 1] = color
    return cv2.addWeighted(image, 1, colored_mask, alpha, 0)

def polygons_to_mask(polygons, image_shape):
    mask = np.zeros(image_shape, dtype=np.uint8)
    for polygon in polygons:
        pts = np.array(polygon, dtype=np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(mask, [pts], color=(1))
    return mask

class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                        'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                        'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                        'magenta', 'sienna', 'maroon']
        self.colors_values = {
                        'blue': (255, 0, 0), 
                        'purple': (128, 0, 128), 
                        'red': (0, 0, 255), 
                        'green': (0, 128, 0), 
                        'orange': (0, 165, 255), 
                        'salmon': (114, 128, 250), 
                        'pink': (203, 192, 255), 
                        'gold': (0, 215, 255),
                        'orchid': (214, 112, 218), 
                        'slateblue': (205, 90, 106), 
                        'limegreen': (50, 205, 50), 
                        'seagreen': (87, 139, 46), 
                        'darkgreen': (0, 100, 0), 
                        'olive': (0, 128, 128),
                        'teal': (128, 128, 0), 
                        'aquamarine': (212, 255, 127), 
                        'steelblue': (180, 130, 70), 
                        'powderblue': (230, 224, 176), 
                        'dodgerblue': (255, 144, 30), 
                        'navy': (128, 0, 0),
                        'magenta': (255, 0, 255), 
                        'sienna': (45, 82, 160), 
                        'maroon': (0, 0, 128)
                    }     
        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        print(self.coco)
        json_file.close()
        
        #self.process_info()
        #self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()

    def display_info(self):
        print('Dataset Info:')
        print('=============')
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))
        
        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')
        
    def display_licenses(self):
        print('Licenses:')
        print('=========')
        
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')
        
    def display_categories(self):
        print('Categories:')
        print('=========')

    
    def display_image(self, image_id, show_polys=True, show_bbox=False, show_labels=False, show_crowds=False, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
        
        image_path = os.path.join(self.image_dir, image['file_name'])
        print(image_path)
        image = Image.open(image_path)
        image = Image.fromarray(visualize_drr(image))
        # image.show()
        # buffered = BytesIO()
        # if image.mode != 'RGB':
        # image = image.convert('L')
    
        max_width = 900
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        image = np.array(image)

        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        labels = {}
        print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            print(segm)
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})
                            
                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})
                                
                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list  
            else:
                for segmentation_points in segm['segmentation']:
                    polygons_list.append(segmentation_points)
                    # segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
                
            polygons[segm['id']] = polygons_list
            
            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'
            
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = bbox_points
            
            labels[segm['id']] = (self.categories[segm['category_id']]['name'], (bbox_points[0] + 15, bbox_points[1] + 15))
            
            # Print details
            print('{}:{}:{}'.format(segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))

        # Draw segmentation polygons on image
        if show_polys:
            print('polygons items')
            print(polygons.items())
            for seg_id, points_list in polygons.items():
                color = poly_colors[seg_id]
                stroke_color = self.colors_values[color]
                segs = [np.array(seg, np.int32).reshape((1, -1, 2))
                        for seg in points_list]
                mask = np.zeros((image_height, image_width))
                mask = cv2.fillPoly(mask, pts = segs, color = (1))
                image = apply_translucent_mask(image, mask, color = stroke_color, alpha=0.2)
        
        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = self.colors_values(poly_colors[seg_id])
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)
        
        if show_labels:
            for seg_id, label in labels.items():
                color = self.colors_values[poly_colors[seg_id]]
                cv2.putText(image, label[0], (label[1][0], label[1][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                color = self.colors_values[poly_colors[seg_id]]
                points = np.array(points, np.int32).reshape((-1, 1, 2))
                cv2.polylines(image, [points], isClosed=True, color=color, thickness=1)

        # if show_bbox:
        #     for seg_id, points in bbox_polygons.items():
        #         fill_color = poly_colors[seg_id]
        #         stroke_color = poly_colors[seg_id]
        #         html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)
                
        # if show_labels:
        #     for seg_id, label in labels.items():
        #         color = poly_colors[seg_id]
        #         html += '<text x="{}" y="{}" style="fill:{}; font-size: 12pt;">{}</text>'.format(label[1][0], label[1][1], color, label[0])
                
        return  image
       
    def process_info(self):
        self.info = self.coco['info']
    
    def process_licenses(self):
        self.licenses = self.coco['licenses']
    
    def process_categories(self):
        self.categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            
            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))
                
    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image
                
    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)
        print(image_id)

In [ ]:
import matplotlib.pyplot as plt
annotation_path = 'coco_annotations_new.json'
image_dir = '/scratch/dr/y.nawar/pengwin/train/input/images/x-ray/'

coco_dataset = CocoDataset(annotation_path, image_dir)
# image = coco_dataset.display_image(3, use_url=False , show_bbox=True , show_labels=True)
# # IPython.display.HTML(html)
# plt.imshow(image)
# plt.show()

In [ ]:
image = coco_dataset.display_image(120, use_url=False , show_bbox=True , show_labels=True)
# IPython.display.HTML(html)
plt.imshow(image)
plt.show()

In [ ]:
import pengwin_utils
from PIL import Image

image_path = "/scratch/dr/y.nawar/pengwin/train/input/images/x-ray/006_0330.tif"
seg_path = "/scratch/dr/y.nawar/pengwin/train/output/images/x-ray/006_0330.tif"

# load image and masks
image = pengwin_utils.load_image(image_path) # raw intensity image
masks, category_ids, fragment_ids = pengwin_utils.load_masks(seg_path)

# save visualization of image and masks
# applies CLAHE normalization to the raw intensity image before overlaying segmentations.
vis_image = pengwin_utils.visualize_sample(image, masks, category_ids, fragment_ids)
vis_path = "vis_image_006_0330.tif.png"
Image.fromarray(vis_image).save(vis_path)
print(f"Wrote visualization to {vis_path}")

# Obtain predicted masks, category_ids, and fragment_ids
# Category IDs are {"SA": 1, "LI": 2, "RI": 3}
# Fragment IDs are the integer labels from label_{category}.nii.gz, with 1 corresponding to the main fragment.
pred_masks, pred_category_ids, pred_fragment_ids = masks, category_ids, fragment_ids # replace with your model

# save the predicted masks for upload to the challenge
# Note: cv2 does not work with uint32 images. It is recommended to use PIL or imageio.v3
# pred_seg = pengwin_utils.masks_to_seg(pred_masks, pred_category_ids, pred_fragment_ids)
# pred_seg_path = "001_0001.png" # ensure dir exists!
# Image.fromarray(pred_seg).save(pred_seg_path)
# print(f"Wrote segmentation to {pred_seg_path}")

In [ ]:
import matplotlib.pyplot as plt
polygon = {'segmentation': [[353, 0, 353, 4, 352, 5, 352, 7, 350, 9, 350, 11, 347, 14, 346, 14, 346, 18, 344, 20, 344, 21, 345, 22, 344, 23, 344, 31, 345, 31, 348, 34, 348, 39, 347, 40, 347, 43, 346, 44, 346, 51, 344, 53, 344, 60, 345, 61, 345, 62, 346, 63, 346, 77, 347, 78, 347, 79, 346, 80, 347, 81, 347, 82, 343, 86, 342, 86, 339, 89, 338, 89, 336, 91, 335, 91, 334, 92, 333, 92, 332, 93, 331, 93, 330, 94, 329, 94, 328, 95, 327, 95, 326, 96, 324, 96, 323, 97, 322, 97, 321, 98, 320, 98, 319, 99, 317, 99, 317, 100, 314, 103, 314, 104, 312, 106, 312, 108, 311, 109, 311, 112, 310, 113, 310, 114, 309, 115, 309, 116, 306, 119, 306, 120, 305, 121, 304, 121, 302, 123, 302, 124, 301, 125, 300, 125, 296, 129, 295, 129, 294, 130, 294, 131, 291, 134, 291, 135, 289, 137, 288, 137, 287, 138, 286, 138, 285, 139, 284, 139, 282, 141, 282, 142, 281, 143, 280, 143, 277, 146, 277, 147, 276, 148, 275, 148, 274, 149, 273, 149, 272, 150, 271, 150, 269, 152, 268, 152, 267, 153, 266, 153, 265, 154, 263, 154, 262, 155, 261, 155, 260, 156, 257, 156, 256, 157, 252, 157, 251, 158, 250, 158, 249, 159, 248, 159, 246, 161, 244, 161, 243, 162, 242, 162, 239, 165, 238, 165, 237, 166, 236, 166, 232, 170, 230, 170, 230, 171, 228, 173, 227, 173, 224, 176, 224, 177, 223, 178, 223, 184, 224, 185, 224, 188, 225, 189, 225, 191, 226, 192, 226, 193, 227, 194, 227, 196, 228, 197, 228, 198, 229, 199, 229, 202, 230, 203, 230, 206, 231, 207, 231, 209, 232, 210, 232, 211, 233, 212, 233, 215, 234, 216, 234, 217, 235, 218, 235, 219, 236, 220, 236, 221, 237, 222, 237, 223, 238, 224, 238, 225, 241, 228, 241, 230, 243, 232, 243, 233, 245, 235, 245, 236, 247, 238, 247, 239, 248, 240, 248, 241, 250, 243, 251, 243, 254, 246, 255, 246, 256, 247, 257, 247, 258, 248, 259, 248, 260, 249, 260, 250, 261, 250, 262, 251, 263, 251, 264, 252, 265, 252, 266, 253, 268, 253, 270, 255, 271, 255, 272, 256, 273, 256, 274, 257, 275, 257, 276, 258, 277, 258, 279, 260, 280, 260, 283, 263, 284, 263, 289, 268, 289, 269, 291, 271, 292, 271, 297, 276, 298, 276, 300, 278, 301, 278, 305, 282, 306, 282, 307, 283, 308, 283, 311, 286, 312, 286, 314, 288, 315, 288, 318, 291, 319, 291, 321, 293, 321, 294, 322, 294, 324, 296, 325, 296, 326, 297, 327, 297, 328, 298, 329, 298, 331, 300, 332, 300, 333, 301, 335, 301, 337, 303, 339, 303, 341, 305, 344, 305, 345, 306, 346, 305, 348, 307, 349, 307, 350, 308, 353, 308, 355, 310, 362, 310, 363, 309, 364, 310, 365, 310, 366, 311, 367, 311, 368, 312, 370, 312, 371, 311, 374, 311, 375, 312, 379, 312, 380, 311, 381, 311, 383, 309, 385, 309, 386, 308, 388, 308, 390, 306, 391, 306, 392, 305, 393, 305, 395, 303, 396, 303, 398, 301, 399, 301, 402, 298, 404, 298, 404, 297, 405, 296, 407, 296, 411, 292, 412, 292, 413, 291, 415, 291, 417, 289, 417, 288, 418, 287, 418, 286, 419, 285, 419, 283, 420, 282, 420, 280, 421, 279, 421, 278, 422, 277, 422, 275, 423, 274, 423, 264, 424, 263, 425, 263, 425, 259, 426, 258, 426, 256, 427, 255, 427, 254, 429, 252, 429, 251, 430, 250, 430, 249, 431, 248, 431, 246, 432, 245, 433, 245, 433, 243, 434, 242, 434, 241, 433, 240, 434, 239, 434, 238, 433, 237, 433, 234, 428, 229, 427, 229, 420, 222, 420, 221, 419, 220, 419, 219, 418, 218, 418, 216, 417, 215, 417, 204, 418, 203, 418, 200, 419, 199, 420, 199, 420, 196, 424, 192, 424, 191, 431, 184, 431, 183, 436, 178, 436, 177, 437, 176, 438, 176, 438, 175, 439, 174, 439, 173, 442, 170, 442, 169, 443, 168, 443, 167, 445, 165, 445, 162, 446, 161, 446, 160, 447, 159, 447, 124, 446, 123, 446, 117, 445, 116, 445, 111, 444, 110, 444, 107, 443, 106, 443, 105, 442, 104, 442, 101, 440, 99, 440, 98, 439, 97, 439, 96, 437, 94, 437, 93, 434, 90, 434, 89, 433, 89, 429, 85, 429, 84, 427, 82, 426, 82, 424, 80, 424, 79, 425, 78, 426, 78, 433, 71, 433, 69, 435, 67, 435, 63, 436, 62, 436, 61, 437, 60, 437, 56, 438, 55, 438, 52, 439, 51, 439, 39, 440, 38, 440, 30, 441, 29, 441, 27, 440, 26, 440, 24, 439, 23, 439, 21, 440, 20, 440, 19, 439, 18, 439, 10, 438, 9, 439, 8, 439, 6, 440, 5, 440, 2, 441, 1, 441, 0], [308, 175, 309, 174, 310, 174, 311, 175, 316, 175, 317, 176, 319, 176, 320, 177, 321, 177, 327, 183, 328, 183, 333, 188, 334, 188, 336, 190, 336, 192, 337, 193, 337, 194, 338, 195, 338, 196, 339, 197, 339, 198, 340, 199, 340, 202, 342, 204, 342, 207, 343, 208, 343, 209, 345, 211, 345, 212, 346, 213, 346, 215, 347, 216, 347, 217, 348, 218, 348, 219, 349, 220, 349, 222, 351, 224, 351, 225, 352, 226, 352, 228, 353, 229, 353, 231, 354, 232, 354, 234, 355, 235, 355, 239, 356, 240, 356, 246, 355, 247, 355, 252, 354, 253, 353, 253, 353, 255, 351, 257, 351, 258, 350, 259, 349, 259, 348, 260, 345, 260, 343, 258, 342, 259, 341, 259, 340, 258, 339, 258, 336, 255, 332, 255, 331, 254, 331, 253, 326, 253, 324, 251, 323, 252, 322, 251, 322, 250, 321, 249, 320, 249, 319, 248, 318, 248, 317, 247, 316, 247, 315, 246, 314, 246, 312, 244, 311, 244, 309, 242, 308, 242, 306, 240, 305, 240, 302, 237, 301, 237, 299, 235, 299, 233, 297, 231, 297, 230, 296, 229, 296, 227, 294, 225, 294, 222, 293, 221, 293, 206, 294, 205, 294, 201, 295, 200, 295, 199, 296, 198, 296, 197, 297, 196, 297, 195, 298, 194, 299, 194, 299, 193, 301, 191, 301, 190, 302, 189, 302, 188, 303, 187, 303, 184, 306, 181, 306, 180, 307, 179, 307, 178, 308, 177]], 'area': 39576, 'iscrowd': 0, 'image_id': 3, 'bbox': [223.0, 0.0, 225.0, 313.0], 'category_id': 3, 'id': 12}

# polygon = {'segmentation': [[63, 0, 62, 1, 56, 1, 55, 2, 52, 2, 51, 3, 40, 3, 39, 4, 38, 3, 36, 3, 35, 4, 34, 3, 30, 3, 29, 4, 25, 4, 24, 5, 14, 5, 13, 6, 6, 6, 5, 7, 3, 7, 2, 8, 0, 8, 0, 172, 1, 172, 2, 173, 3, 173, 4, 174, 4, 175, 5, 174, 6, 175, 6, 176, 7, 176, 8, 177, 9, 177, 10, 178, 11, 178, 12, 179, 12, 180, 13, 180, 14, 181, 15, 181, 17, 183, 18, 183, 19, 184, 20, 184, 21, 185, 21, 186, 22, 186, 24, 188, 25, 188, 26, 189, 27, 189, 29, 191, 30, 191, 31, 192, 31, 193, 32, 193, 35, 196, 36, 196, 37, 197, 37, 198, 41, 202, 41, 203, 42, 204, 42, 205, 43, 206, 43, 215, 42, 216, 42, 217, 41, 218, 41, 219, 40, 220, 40, 221, 41, 222, 41, 232, 42, 233, 41, 234, 41, 236, 42, 237, 42, 238, 43, 239, 43, 241, 44, 242, 44, 244, 45, 245, 45, 247, 46, 248, 46, 249, 47, 250, 47, 251, 48, 252, 49, 252, 50, 253, 50, 255, 51, 256, 51, 258, 52, 259, 52, 260, 53, 261, 54, 261, 55, 262, 55, 264, 56, 265, 57, 265, 58, 266, 58, 268, 59, 269, 59, 270, 60, 270, 62, 272, 62, 273, 63, 273, 64, 274, 64, 275, 65, 276, 65, 277, 66, 277, 67, 278, 67, 280, 68, 281, 68, 283, 69, 284, 70, 284, 71, 285, 71, 287, 72, 288, 72, 289, 73, 289, 74, 290, 74, 291, 75, 291, 76, 292, 76, 293, 77, 294, 77, 295, 78, 295, 79, 296, 80, 296, 81, 297, 81, 298, 82, 299, 83, 299, 84, 300, 84, 301, 86, 301, 90, 305, 91, 305, 92, 306, 92, 307, 93, 307, 94, 308, 98, 308, 99, 309, 100, 309, 101, 310, 103, 310, 104, 311, 105, 311, 106, 310, 107, 311, 113, 311, 114, 312, 127, 312, 128, 313, 132, 313, 133, 314, 135, 314, 136, 315, 138, 315, 140, 317, 141, 317, 142, 318, 142, 319, 143, 319, 145, 321, 145, 322, 147, 324, 147, 325, 148, 326, 148, 327, 149, 328, 149, 332, 150, 333, 149, 334, 149, 340, 150, 341, 150, 343, 151, 344, 151, 345, 153, 347, 154, 347, 156, 349, 156, 351, 157, 352, 157, 353, 158, 354, 159, 353, 165, 359, 165, 360, 166, 361, 166, 362, 174, 370, 174, 371, 177, 374, 178, 374, 181, 377, 184, 377, 185, 378, 186, 378, 187, 379, 188, 379, 189, 380, 197, 380, 198, 379, 199, 379, 200, 380, 201, 379, 212, 379, 213, 378, 215, 378, 216, 377, 218, 377, 218, 376, 220, 374, 220, 373, 221, 372, 222, 372, 224, 370, 225, 370, 228, 367, 229, 367, 231, 365, 231, 364, 232, 363, 233, 363, 235, 361, 235, 360, 236, 359, 237, 359, 237, 356, 238, 355, 238, 353, 239, 352, 239, 351, 240, 350, 240, 347, 241, 346, 241, 343, 242, 342, 242, 335, 243, 334, 243, 322, 244, 321, 244, 320, 243, 319, 243, 317, 242, 316, 242, 314, 239, 311, 237, 311, 236, 310, 235, 310, 234, 309, 233, 310, 232, 309, 230, 309, 229, 310, 227, 310, 226, 311, 225, 310, 225, 309, 224, 309, 222, 311, 217, 311, 216, 312, 215, 311, 214, 311, 213, 312, 212, 311, 211, 311, 210, 310, 209, 311, 208, 311, 207, 312, 206, 312, 204, 310, 203, 310, 202, 309, 202, 304, 203, 303, 203, 302, 202, 301, 203, 300, 203, 298, 204, 297, 204, 296, 205, 295, 205, 293, 206, 292, 205, 291, 205, 289, 204, 288, 204, 287, 203, 286, 203, 285, 201, 283, 201, 282, 200, 281, 200, 280, 193, 273, 193, 272, 192, 272, 190, 270, 189, 270, 187, 268, 187, 267, 186, 267, 185, 266, 185, 265, 180, 260, 180, 259, 177, 256, 177, 254, 176, 253, 176, 252, 174, 250, 174, 248, 173, 247, 173, 245, 172, 244, 172, 241, 171, 240, 171, 238, 170, 237, 170, 235, 169, 234, 169, 233, 167, 231, 167, 228, 165, 226, 165, 225, 164, 224, 164, 222, 162, 220, 162, 219, 161, 218, 162, 217, 162, 216, 161, 215, 161, 213, 162, 212, 162, 208, 163, 207, 164, 208, 165, 208, 166, 209, 170, 209, 171, 210, 172, 210, 173, 211, 176, 211, 177, 212, 178, 212, 179, 213, 180, 212, 181, 213, 183, 213, 184, 212, 186, 212, 187, 211, 189, 211, 190, 210, 191, 210, 192, 209, 199, 209, 200, 208, 201, 209, 202, 208, 204, 208, 205, 207, 206, 207, 206, 199, 205, 198, 205, 192, 206, 191, 206, 189, 208, 187, 208, 185, 209, 184, 210, 184, 211, 183, 213, 183, 214, 182, 215, 182, 217, 180, 218, 180, 223, 175, 225, 175, 226, 174, 227, 174, 228, 173, 228, 172, 231, 169, 231, 168, 233, 166, 232, 165, 233, 164, 233, 163, 234, 162, 234, 161, 235, 160, 235, 159, 236, 158, 235, 157, 235, 155, 234, 154, 234, 153, 233, 152, 233, 151, 231, 149, 229, 149, 228, 148, 227, 148, 225, 146, 225, 145, 224, 144, 224, 142, 223, 141, 223, 138, 222, 137, 222, 136, 223, 135, 223, 133, 221, 131, 221, 130, 220, 129, 220, 128, 219, 127, 219, 125, 218, 124, 218, 123, 215, 120, 215, 119, 214, 118, 213, 118, 211, 116, 211, 115, 210, 114, 209, 114, 208, 113, 207, 113, 206, 112, 205, 112, 199, 106, 198, 106, 197, 105, 197, 104, 196, 104, 192, 100, 192, 99, 191, 99, 184, 92, 184, 91, 180, 87, 180, 85, 179, 84, 178, 84, 177, 83, 177, 82, 176, 81, 176, 79, 175, 78, 174, 78, 171, 75, 171, 74, 168, 71, 168, 70, 164, 66, 163, 66, 162, 65, 162, 64, 161, 64, 160, 63, 159, 63, 151, 55, 151, 54, 146, 49, 145, 49, 143, 47, 142, 47, 140, 45, 139, 45, 138, 44, 137, 44, 136, 43, 136, 42, 135, 42, 132, 39, 131, 39, 129, 37, 127, 37, 126, 36, 126, 35, 125, 34, 124, 34, 123, 33, 122, 33, 121, 32, 120, 32, 116, 28, 115, 28, 114, 27, 113, 27, 112, 26, 111, 26, 110, 25, 109, 25, 106, 22, 104, 22, 103, 21, 102, 21, 101, 20, 100, 20, 98, 18, 97, 18, 95, 16, 94, 16, 93, 15, 92, 15, 91, 14, 89, 14, 87, 12, 86, 12, 82, 8, 82, 7, 81, 6, 81, 4, 80, 3, 79, 3, 78, 2, 77, 2, 75, 0]], 'area': 55282, 'iscrowd': 0, 'image_id': 3, 'bbox': [0.0, 0.0, 245.0, 381.0], 'category_id': 2, 'id': 10}
polygon = polygon['segmentation']
image_path = '/scratch/dr/y.nawar/pengwin/train/input/images/x-ray/001_0002.tif'
print(polygon)
# Reading an image in default
# mode
image = Image.open(image_path)
image = visualize_drr(image)
 
# Window name in which image is
# displayed
window_name = 'Image'
 
# Polygon corner points coordinates
pts = np.array([[25, 70], [25, 160], 
                [110, 200], [200, 160], 
                [200, 70], [110, 20]],
               np.int32)
isClosed = True
color = (0, 255, 0)
thickness = 1
segs = [np.array(seg, np.int32).reshape((1, -1, 2))
        for seg in polygon]
# print(segs)

pts = pts.reshape((-1, 1, 2))
# print([pts])
image = cv2.polylines(image, segs, 
                      isClosed, color, thickness)
plt.imshow(image)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
annotation_path = '/scratch/dr/y.nawar/pengwin/train/coco_annotations.json'
image_dir = '/scratch/dr/y.nawar/pengwin/train/input/images/x-ray/'

coco_dataset = CocoDataset(annotation_path, image_dir)
# image = coco_dataset.display_image(101, use_url=False , show_bbox=True , show_labels=True)
# IPython.display.HTML(html)
# plt.imshow(image)
# plt.show()
for i in range(50000):
    image = coco_dataset.display_image(i, use_url=False , show_bbox=True , show_labels=True)
    plt.imshow(image)
    plt.show()
    

In [ ]:
import base64
import IPython
import json
import numpy as np
import os
import random
import requests
from io import BytesIO
from math import trunc
from PIL import Image 
from PIL import ImageDraw as PILImageDraw
from pengwin_utils import visualize_drr

def apply_translucent_mask(image, mask, color, alpha=0.5):
    colored_mask = np.zeros_like(image)
    colored_mask[mask == 1] = color
    return cv2.addWeighted(image, 1, colored_mask, alpha, 0)

def polygons_to_mask(polygons, image_shape):
    mask = np.zeros(image_shape, dtype=np.uint8)
    for polygon in polygons:
        pts = np.array(polygon, dtype=np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(mask, [pts], color=(1))
    return mask

class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                        'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                        'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                        'magenta', 'sienna', 'maroon']
        self.colors_values = {
                        'blue': (255, 0, 0), 
                        'purple': (128, 0, 128), 
                        'red': (0, 0, 255), 
                        'green': (0, 128, 0), 
                        'orange': (0, 165, 255), 
                        'salmon': (114, 128, 250), 
                        'pink': (203, 192, 255), 
                        'gold': (0, 215, 255),
                        'orchid': (214, 112, 218), 
                        'slateblue': (205, 90, 106), 
                        'limegreen': (50, 205, 50), 
                        'seagreen': (87, 139, 46), 
                        'darkgreen': (0, 100, 0), 
                        'olive': (0, 128, 128),
                        'teal': (128, 128, 0), 
                        'aquamarine': (212, 255, 127), 
                        'steelblue': (180, 130, 70), 
                        'powderblue': (230, 224, 176), 
                        'dodgerblue': (255, 144, 30), 
                        'navy': (128, 0, 0),
                        'magenta': (255, 0, 255), 
                        'sienna': (45, 82, 160), 
                        'maroon': (0, 0, 128)
                    }     
        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        print(self.coco)
        json_file.close()
        
        #self.process_info()
        #self.process_licenses()
        self.process_categories()
        print('finished cats')
        self.process_images()
        print('finished images')
        self.process_segmentations()
        print('finished segmentations')

    def display_info(self):
        print('Dataset Info:')
        print('=============')
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))
        
        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')
        
    def display_licenses(self):
        print('Licenses:')
        print('=========')
        
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')
        
    def display_categories(self):
        print('Categories:')
        print('=========')

    
    def display_image(self, image_id, show_polys=True, show_bbox=False, show_labels=False, show_crowds=False, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
        
        image_path = os.path.join(self.image_dir, image['file_name'])
        print(image_path)
        image = Image.open(image_path)
        image = Image.fromarray(visualize_drr(image))
        # image.show()
        # buffered = BytesIO()
        # if image.mode != 'RGB':
        # image = image.convert('L')
    
        max_width = 900
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        image = np.array(image)

        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        labels = {}
        print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            print(segm)
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})
                            
                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})
                                
                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list  
            else:
                for segmentation_points in segm['segmentation']:
                    polygons_list.append(segmentation_points)
                    # segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
                
            polygons[segm['id']] = polygons_list
            
            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'
            
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = bbox_points
            
            labels[segm['id']] = (self.categories[segm['category_id']]['name'], (bbox_points[0] + 15, bbox_points[1] + 15))
            
            # Print details
            print('{}:{}:{}'.format(segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))

        # Draw segmentation polygons on image
        if show_polys:
            print('polygons items')
            print(polygons.items())
            for seg_id, points_list in polygons.items():
                color = poly_colors[seg_id]
                stroke_color = self.colors_values[color]
                segs = [np.array(seg, np.int32).reshape((1, -1, 2))
                        for seg in points_list]
                mask = np.zeros((image_height, image_width))
                mask = cv2.fillPoly(mask, pts = segs, color = (1))
                image = apply_translucent_mask(image, mask, color = stroke_color, alpha=0.2)
        
        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = self.colors_values(poly_colors[seg_id])
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)
        
        if show_labels:
            for seg_id, label in labels.items():
                color = self.colors_values[poly_colors[seg_id]]
                cv2.putText(image, label[0], (label[1][0], label[1][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                color = self.colors_values[poly_colors[seg_id]]
                points = np.array(points, np.int32).reshape((-1, 1, 2))
                cv2.polylines(image, [points], isClosed=True, color=color, thickness=1)
                
        return  image
       
    def process_info(self):
        self.info = self.coco['info']
    
    def process_licenses(self):
        self.licenses = self.coco['licenses']
    
    def process_categories(self):
        self.categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            
            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))
                
    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image
                
    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)
            print(image_id)
            print(segmentation)


annotation_path = '/scratch/dr/y.nawar/pengwin/train/coco_annotations.json'
image_dir = '/scratch/dr/y.nawar/pengwin/train/input/images/x-ray/'

coco_dataset = CocoDataset(annotation_path, image_dir)